In [1]:
import numpy as np
import sys
import pandas as pd 
from keras.layers import Input, Lambda, Dense, Flatten , Dropout , MaxPool2D
from keras.models import Model , Sequential
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from glob import glob
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import random
import os
from distutils.file_util import copy_file
import warnings
warnings.filterwarnings('ignore')
from keras.callbacks import EarlyStopping ,ReduceLROnPlateau

In [2]:
batch_size=64

train_data_gen = ImageDataGenerator(rescale=1./255 ,
                                    zoom_range=0.3, 
                                    horizontal_flip=True, 
                                    shear_range=0.2,
                                    rotation_range=30
                                    )

train = train_data_gen.flow_from_directory(directory='/home/mrk9fx/dl_data/Data/Train',
                                           class_mode ='categorical',
                                           batch_size=64,
                                           target_size=(224,224))  

Found 11449 images belonging to 3 classes.


In [3]:
test_data_gen = ImageDataGenerator(rescale=1./255 )

test = test_data_gen.flow_from_directory(directory='/home/mrk9fx/dl_data/Data/Test', 
                                         target_size=(224,224), 
                                         class_mode='categorical',
                                         shuffle=False,
                                         batch_size=64)

valid = test_data_gen.flow_from_directory(directory='/home/mrk9fx/dl_data/Data/Valid', 
                                          target_size=(224,224), 
                                          class_mode = 'categorical',
                                          shuffle=False,
                                          batch_size=64)

Found 2170 images belonging to 3 classes.
Found 1535 images belonging to 3 classes.


In [4]:
from tensorflow import keras
#model = tf.keras.applications.DenseNet121(include_top = False,
    #                                       weights = 'imagenet',
    #                                       input_shape = (224,224,3), )
model = tf.keras.applications.DenseNet201(include_top = False,
                                           weights = 'imagenet',
                                           input_shape = (224,224,3), )

In [5]:
#for layer in inc.layers:           
 # layer.trainable = False

In [6]:
#Definition of the Keras model outputs

#x = Flatten()(inc.output)
#prediction= Dense(units=1, activation='sigmoid')(x)

#model = Model(inc.input,prediction)


n_classes = 3

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

# Rebuild top
x = tf.keras.layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
x = tf.keras.layers.BatchNormalization()(x)

top_dropout_rate = 0.2
x = tf.keras.layers.Dropout(top_dropout_rate, name="top_dropout")(x)
# x = tf.keras.layers.Flatten()(x)
outputs = tf.keras.layers.Dense(3, activation="softmax", name="pred")(x) # match number of classes

model_2 = keras.models.Model(inputs=model.input,
                           outputs=outputs)

#model.add(tf.keras.layers.Dense(3, activation='softmax', name='pred'))


In [7]:
#Compile model

#model_2.compile(
 #   optimizer = tf.keras.optimizers.Adam(0.001),
 #   loss = 'binary_crossentropy',
 #   metrics = ['accuracy'],
#)

for layer in model_2.layers:
    layer.trainable = False

recall = tf.keras.metrics.Recall
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01, decay=0.01)
model_2.compile(optimizer=optimizer,loss="categorical_crossentropy",
              metrics=['accuracy'],)

In [8]:
early = EarlyStopping(monitor="val_accuracy", mode='min', patience=10)
learning_rate_reduction = ReduceLROnPlateau(monitor="val_accuracy", patience=2, verbose=1, factor=0.3, min_lr=0.0001)
callbacks_list = [early, learning_rate_reduction]

In [9]:
from sklearn.utils.class_weight import compute_class_weight

weights = compute_class_weight(class_weight="balanced",
                               classes=np.unique(train.classes), 
                               y=train.classes)
cw = dict(zip(np.unique(train.classes), weights))
print(cw)

{0: 1.3191611936859085, 1: 0.46803204970975393, 2: 9.493366500829188}


In [10]:
hist = model_2.fit_generator(train, 
                 steps_per_epoch=len(train)//3, 
                 epochs=10,
                 validation_data=valid, 
                 class_weight=cw,
                 validation_steps=len(test)//3,
                 callbacks=callbacks_list
                )

Epoch 1/10
59/59 [==============================] - 258s 4s/step - loss: 1.4794 - accuracy: 0.2055 - val_loss: 1.4057 - val_accuracy: 0.4304 - lr: 0.0100
Epoch 2/10
59/59 [==============================] - 240s 4s/step - loss: 1.5299 - accuracy: 0.1990 - val_loss: 1.4057 - val_accuracy: 0.4304 - lr: 0.0100
Epoch 3/10
59/59 [==============================] - ETA: 0s - loss: 1.5742 - accuracy: 0.1905
Epoch 3: ReduceLROnPlateau reducing learning rate to 0.0029999999329447745.
59/59 [==============================] - 242s 4s/step - loss: 1.5742 - accuracy: 0.1905 - val_loss: 1.4057 - val_accuracy: 0.4304 - lr: 0.0100
Epoch 4/10
59/59 [==============================] - 246s 4s/step - loss: 1.4875 - accuracy: 0.1888 - val_loss: 1.4057 - val_accuracy: 0.4304 - lr: 0.0030
Epoch 5/10
59/59 [==============================] - ETA: 0s - loss: 1.5637 - accuracy: 0.1881
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.0009000000078231095.
59/59 [==============================] - 245s 4s/step -

In [11]:
model_2.save('model_Dense_2')

INFO:tensorflow:Assets written to: model_Dense_2/assets
